# Sunspot Detection with Python ,OpenCV and Astropy on FITS files

In [ ]:
import cv2
import numpy as np
from astropy.io import fits


def find_sun(filename):
  # Load the image and normalize it (0-255)
  image = fits.getdata(filename, ext=0)
  image = (image / np.max(image) * 255).astype(np.uint8)

  # Bluring the image to reduce noise
  blurred = cv2.GaussianBlur(image, (5, 5), 0)

  # Detect edges using Canny edge detection with adjusted thresholds
  edges = cv2.Canny(blurred, 30, 100)

  # Find circles in the edge image with adjusted parameters for better detection
  circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, dp=1.2, minDist=100,
                             param1=30, param2=20, minRadius=100, maxRadius=300)

  # Check if a sun is found
  if circles is not None:
    # first circle (assuming it's the sun) and converting it to integers
    circles = np.round(circles[0, :]).astype("int")
    x, y, r = circles[0]  # unpack circle parameters
    print(f"Sun detected at ({x}, {y}) with radius {r}")

    # Creating a color image from blurred greyscale image
    color_image = cv2.cvtColor(blurred, cv2.COLOR_GRAY2BGR)

    # Drawing the sun circle on the color image
    cv2.circle(color_image, (x, y), r, (0, 255, 0), 4)

    # Saving the image with the sun circle
    cv2.imwrite('detected_sun.png', color_image)
  else:
    print("No sun detected")
    # Saving the blurred image
    cv2.imwrite('blurred_image.png', blurred)

  # Creating binary image to highlight sunspots
  _, binary_sunspots = cv2.threshold(blurred, 50, 255, cv2.THRESH_BINARY_INV)
  # Saving binary image
  cv2.imwrite('binary_sunspots.png', binary_sunspots)


# Set the filename of your FITS image
image_file = 'file1.fits'
find_sun(image_file)

# Sunspot Detection on Images with Python and Matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread


def process_image(image_path):
    # Read the image (assuming a grayscale image for simplicity)
    image = imread(image_path)
    if len(image.shape) == 3:  # If the image is RGB, convert to grayscale
        image = np.mean(image[:, :, :3], axis=2)

    # Normalize the image
    normalized_image = image / np.max(image)

    # Threshold the image to detect darker spots (e.g., sunspots)
    # Adjust the threshold value based on your specific images
    threshold_value = 0.5
    binary_image = normalized_image < threshold_value

    # Convert boolean array to int
    binary_image = binary_image.astype(int)

    # Display the original and binary images
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(normalized_image, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(binary_image, cmap='gray')
    plt.title('Binary Image')
    plt.axis('off')

    plt.show()

    return binary_image


# Example usage
image_path = 'detected_sun.png'
binary_image = process_image(image_path)